In [1]:
import numpy as np
import gprob as gp

# Introduction

 Random variables in gprob come in arrays. In many ways, they behave like numpy arrays, and often can be handled via methods and functions with the same names as in numpy.

A multi-dimensional random variable, such as x,

In [2]:
x = gp.normal(size=(1, 2, 3))
x

Normal(
  mean=[[[0. 0. 0.]
         [0. 0. 0.]]]
   var=[[[1. 1. 1.]
         [1. 1. 1.]]]
)

It has a shape, number of dimensions, and size

In [3]:
print(x.shape)
print(x.ndim)
print(x.size)

(1, 2, 3)
3
6


The variable's shape is the shape of its mean, varaince, and samples

In [4]:
print(x.mean().shape)
print(x.var().shape)
print(x.sample().shape)

(1, 2, 3)
(1, 2, 3)
(1, 2, 3)


The covariance of the variable has twice this shape

In [5]:
print(x.cov().shape)

(1, 2, 3, 1, 2, 3)


For the variable we created, the covariance function reshaped to 2D is an identity matrix

In [6]:
np.reshape(x.cov(), (x.size, x.size))

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

# Shapes

Variables can be reshaped or flattened,

In [7]:
x = gp.normal(size=(1, 2, 4))

x = x.ravel() + np.arange(8)  # Flattening and adding mean.
x = gp.reshape(x, (2, 2, 2))

x.mean()

array([[[0., 1.],
        [2., 3.]],

       [[4., 5.],
        [6., 7.]]])

and indexed,

In [8]:
y = x[1, 0, :]

gp.cov(y, x[1, 0, :])

array([[1., 0.],
       [0., 1.]])

Their dimensions can be expanded using `None` key (aka `np.newaxis`),

In [9]:
x = gp.normal(size=(2,))

y = x[..., None]
y.shape

(2, 1)

or shrunk

In [10]:
x = gp.normal(size=(2, 1))

y = x.squeeze(axis=-1)
y.shape

(2,)

# Broadcasting

Variables of different shapes can be combined in element-wise operations when they are broadcastable according to numpy's rules. (Basically, saying that the shape of the lower-dimensional variable can be mismatched with the end of the shape of the higher-dimensional variable only where one of their dimensions is of size 1).

Broadcasting in the addition of two varaibles can look like

In [11]:
x = gp.normal(size=2)  # A vector with the shape (2,).
y = gp.normal()  # A scalar.

z = x + y  # This adds y to both elements of x.

gp.cov(gp.hstack([z, y]))

array([[2., 1., 1.],
       [1., 2., 1.],
       [1., 1., 1.]])

and broadcasting in the multiplication of a variable by a constant can look like

In [12]:
x = gp.normal(size=2)  # A vector with the shape (2,).
y = np.array([[1, 2], [3, 4]]) * x  # A matrix with the shape (2, 2).

gp.cov(x, y)

array([[[1., 0.],
        [3., 0.]],

       [[0., 2.],
        [0., 4.]]])

# Complex variables

Random variables can be complex-valued,

In [13]:
x = gp.normal(size=(2,)) + 1j * gp.normal(size=(2,))
x

Normal(mean=[0.+0.j 0.+0.j], var=[2. 2.])

The covariance of a complex variable is the expectation of it times its conjugate,

In [14]:
x = gp.normal()
y = gp.normal() + 2j * x

xy = gp.stack([x, y])
xy.cov()

array([[1.+0.j, 0.-2.j],
       [0.+2.j, 5.+0.j]])

Defined in this way, the covariance of a multi-dimensional complex variable does not fully specify its correlations in general. (It can fully specify the correlations, e.g. if the variable is *circular*.) Complete information about a complex variable is contained in its mean and the covariance of its real and imaginary parts.  

In [15]:
xy_real = gp.concatenate([xy.real, xy.imag])
xy_real.cov()

array([[1., 0., 0., 2.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [2., 0., 0., 4.]])

# Vector operations

Random variables can be used in vector operations, such as matrix multiplication, Einstein summation, or inner and outer products. 

For example, the scalar product of a random vector and a deterministic vector can be calculated as

In [16]:
x = gp.normal(size=3)
y = np.array([1, 2, 3])

z = gp.dot(x, y)  # z = x[0] * y[0] + x[1] * y[1] + x[2] * y[2]
z

Normal(mean=0, var=14)

In [17]:
gp.cov(z, x)

array([1., 2., 3.])

Another example is the matrix-vector product

In [18]:
x = gp.normal(size=2)
y = np.array([[1, 2], [3, 4]])

y @ x

Normal(mean=[0. 0.], var=[ 5. 25.])

which can be alco calculated as

In [19]:
gp.einsum("ij, j -> i", y, x)

Normal(mean=[0. 0.], var=[ 5. 25.])

# fft and linalg

Among numpy's sub-modules, gprob reproduces `fft` and `linalg`. E.g., random variables can be Fourier-transformed as

In [20]:
x = gp.normal(0.1, 1, size=4) * np.array([1, 2, 3, 4])
gp.fft.fft(x)

Normal(mean=[ 1. +0.j  -0.2+0.2j -0.2+0.j  -0.2-0.2j], var=[30. 30. 30. 30.])

`linalg` now mostly consists of `solve` function (and its twin `asolve` that supports batching), which is a more economic way to calculate products of inverse matrices and vectors,

In [21]:
x = gp.normal(size=2)
y = np.array([[1, 2], [3, 4]])

gp.linalg.solve(y, x)  # The same result as np.linalg.inv(y) @ x

Normal(mean=[0. 0.], var=[5.  2.5])

# Linearized operations

All operations on arrays of random variables so far have been linear in terms of the transformations of their latent spaces, and therefore produced Gaussian random variables as their results. An operation like multiplication of two Gaussian variables creates a varaible that is distributed according to a non-Gaussian law. gprob does not operate with such distributions.

So, what happens when two varaibles are multiplied? One obvious choice could be making such operations forbidden. Another choice would be to make them output something that *is* a Gaussian distribution as a result. The second is what is done in gprob. Nonlinear operations produce linearized results.

E.g.,

In [22]:
x = gp.normal(5, 0.5)
y = gp.normal(4, 0.3)

x * y

Normal(mean=20, var=15.5)

which is the same as

In [23]:
x0 = x.mean()
y0 = y.mean()

dx = x - x0
dy = y - y0

x0 * y0 + dx * y0 + dy * x0

Normal(mean=20, var=15.5)

The linearization is done via internal forward-mode atomatic differentiation.

# Item assignments

Random variables support item assignment,

In [24]:
x = gp.normal(size=3)
x[1] = 2 * x[0] + x[1] + 1

x.cov()

array([[1., 2., 0.],
       [2., 5., 0.],
       [0., 0., 1.]])

This is rarely an efficient way to manipulate distributions, though. Maybe except for examples like the one above, where all the variables in the expression on the right-hand side come from the distribution on the left-hand side.

Also, there is a caveat. Assignments can have side effects on variables that the assignment target was derived from. Or not have side effects, depending on the way it was derived. This is similar to numpy arrays: if the operations that created the array whose elements are assigned to returned views, the original arrays are modified upon assignment, and if the operations created copies, the originals are not modified. In gprob, operations return copies more often than in numpy.

Consider

In [25]:
x = gp.normal(size=4).ravel()

y = x[:2]  # A view of x.
x[1] += x[0]

y.cov()  # y was modified as a side effect of setting x.

array([[1., 1.],
       [1., 2.]])

but

In [26]:
x = gp.normal(size=4).ravel()

y = x[:2]  # A view of x.
x[1] += x[0] + gp.normal()  # Adding a new random variable triggers copying.

y.cov()  # y was not modified; setting x had no side effects.

array([[1., 0.],
       [0., 1.]])